## Hour

In [267]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import plotly.express as px


file_path = '/home/olga/ts_year_project/TimeSeriesMasters2023/eda_olga/energy_hourly_dataset_2012.csv'
df = pd.read_csv(file_path)
df

,date,id,value
0,2012-01-01 00:00:01,1,69.0
1,2012-01-01 01:00:01,2,92.0
2,2012-01-01 02:00:01,3,96.0
3,2012-01-01 03:00:01,4,92.0
4,2012-01-01 04:00:01,5,91.0
...,...,...,...
2179,2012-03-31 19:00:01,2180,110.0
2180,2012-03-31 20:00:01,2181,116.0
2181,2012-03-31 21:00:01,2182,105.0
2182,2012-03-31 22:00:01,2183,95.0


In [268]:
def transform_to_custom_freq(df, freq):
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    # df.set_index('date', inplace=True)
    df_resampled = df.resample(freq).sum()
    # df_resampled.reset_index(inplace=True)
    
    return df_resampled

## week

In [269]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
selected_freq = 'W'
df_resampled_week = transform_to_custom_freq(df, selected_freq)
df_resampled_week

,id,value
date,,
2012-01-01,300,2556.0
2012-01-08,18228,18867.0
2012-01-15,46452,17953.0
2012-01-22,74676,17975.0
2012-01-29,102900,18178.0
2012-02-05,131124,17876.0
2012-02-12,159348,18309.0
2012-02-19,187572,18519.0
2012-02-26,215796,19173.0


## Daily

In [270]:
df.reset_index(inplace=True)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
selected_freq = 'D'
df_resampled_day = transform_to_custom_freq(df, selected_freq)
df_resampled_day

,id,value
date,,
2012-01-01,300,2556.0
2012-01-02,876,2635.0
2012-01-03,1452,2928.0
2012-01-04,2028,2567.0
2012-01-05,2604,2700.0
...,...,...
2012-03-27,49836,2794.0
2012-03-28,50412,2760.0
2012-03-29,50988,2260.0


## Month

In [271]:
df.reset_index(inplace=True)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
# df.reset_index(inplace=True)
selected_freq = 'M'
df_resampled_month = transform_to_custom_freq(df, selected_freq)
df_resampled_month

,id,value
date,,
2012-01-31,277140,80686.0
2012-02-29,760380,76964.0
2012-03-31,1348500,80791.0


In [272]:
from hierarchicalforecast.utils import aggregate
from hierarchicalforecast.methods import BottomUp, TopDown, MinTrace, ERM
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation

## Full aggregation

In [273]:
df

,id,value
date,,
2012-01-01 00:00:01,1,69.0
2012-01-01 01:00:01,2,92.0
2012-01-01 02:00:01,3,96.0
2012-01-01 03:00:01,4,92.0
2012-01-01 04:00:01,5,91.0
...,...,...
2012-03-31 19:00:01,2180,110.0
2012-03-31 20:00:01,2181,116.0
2012-03-31 21:00:01,2182,105.0


In [274]:
# df_reduced = df.drop(columns=['index', 'id'])
df_reduced = df

In [275]:
df_reduced.reset_index(inplace=True)
df_reduced['date'] = pd.to_datetime(df_reduced['date'])
# Добавление столбцов 'hour', 'day' и 'week'
df_reduced['hour'] = df_reduced['date'].astype(str)
df_reduced['day'] = df_reduced['date'].dt.strftime('%Y-%m-%d %H:%M:%S').apply(lambda x: x.split(' ')[0]).astype(str)
df_reduced['week'] = df_reduced['date'].dt.to_period("W-SUN").apply(lambda x: x.start_time).astype(str)
df_reduced['month'] = df_reduced['date'].dt.strftime('%Y-%m').astype(str)

# Вывод результата
df_reduced

,date,id,value,hour,day,week,month
0,2012-01-01 00:00:01,1,69.0,2012-01-01 00:00:01,2012-01-01,2011-12-26,2012-01
1,2012-01-01 01:00:01,2,92.0,2012-01-01 01:00:01,2012-01-01,2011-12-26,2012-01
2,2012-01-01 02:00:01,3,96.0,2012-01-01 02:00:01,2012-01-01,2011-12-26,2012-01
3,2012-01-01 03:00:01,4,92.0,2012-01-01 03:00:01,2012-01-01,2011-12-26,2012-01
4,2012-01-01 04:00:01,5,91.0,2012-01-01 04:00:01,2012-01-01,2011-12-26,2012-01
...,...,...,...,...,...,...,...
2179,2012-03-31 19:00:01,2180,110.0,2012-03-31 19:00:01,2012-03-31,2012-03-26,2012-03
2180,2012-03-31 20:00:01,2181,116.0,2012-03-31 20:00:01,2012-03-31,2012-03-26,2012-03
2181,2012-03-31 21:00:01,2182,105.0,2012-03-31 21:00:01,2012-03-31,2012-03-26,2012-03
2182,2012-03-31 22:00:01,2183,95.0,2012-03-31 22:00:01,2012-03-31,2012-03-26,2012-03


In [276]:
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2184 entries, 0 to 2183
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    2184 non-null   datetime64[ns]
 1   id      2184 non-null   int64         
 2   value   2184 non-null   float64       
 3   hour    2184 non-null   object        
 4   day     2184 non-null   object        
 5   week    2184 non-null   object        
 6   month   2184 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 119.6+ KB


In [277]:
df_reduced = df_reduced.rename(columns={'value': 'y', 'date': 'ds'})
# Create hierarchical structure and constraints
hierarchy_levels = [
    ['month'],
    ['month', 'week'],
    ['month', 'week', 'day'],
    ['month', 'week', 'day', 'hour']
]
Y_hier_df, S_df, tags = aggregate(df=df_reduced, spec=hierarchy_levels)
Y_hier_df = Y_hier_df.reset_index()
print('S_df.shape', S_df.shape)
print('Y_hier_df.shape', Y_hier_df.shape)
print("tags['month']", tags['month'])

S_df.shape (2294, 2184)
Y_hier_df.shape (8736, 3)
tags['month'] ['2012-01' '2012-02' '2012-03']


/home/olga/hse_master/develop_tools/fast_api/fast_api_tutorial/venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [278]:
tags

{'month': array(['2012-01', '2012-02', '2012-03'], dtype=object),
 'month/week': array(['2012-01/2011-12-26', '2012-01/2012-01-02', '2012-01/2012-01-09',
        '2012-01/2012-01-16', '2012-01/2012-01-23', '2012-01/2012-01-30',
        '2012-02/2012-01-30', '2012-02/2012-02-06', '2012-02/2012-02-13',
        '2012-02/2012-02-20', '2012-02/2012-02-27', '2012-03/2012-02-27',
        '2012-03/2012-03-05', '2012-03/2012-03-12', '2012-03/2012-03-19',
        '2012-03/2012-03-26'], dtype=object),
 'month/week/day': array(['2012-01/2011-12-26/2012-01-01', '2012-01/2012-01-02/2012-01-02',
        '2012-01/2012-01-02/2012-01-03', '2012-01/2012-01-02/2012-01-04',
        '2012-01/2012-01-02/2012-01-05', '2012-01/2012-01-02/2012-01-06',
        '2012-01/2012-01-02/2012-01-07', '2012-01/2012-01-02/2012-01-08',
        '2012-01/2012-01-09/2012-01-09', '2012-01/2012-01-09/2012-01-10',
        '2012-01/2012-01-09/2012-01-11', '2012-01/2012-01-09/2012-01-12',
        '2012-01/2012-01-09/2012-01-13', '

In [279]:
S_df

,2012-01/2011-12-26/2012-01-01/2012-01-01 00:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 01:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 02:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 03:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 04:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 05:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 06:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 07:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 08:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 09:00:01,...,2012-03/2012-03-26/2012-03-31/2012-03-31 14:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 15:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 16:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 17:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 18:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 19:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 20:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 21:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 22:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 23:00:01
2012-01,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2012-01/2011-12-26,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-01/2012-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-03/2012-03-26/2012-03-31/2012-03-31 19:00:01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2012-03/2012-03-26/2012-03-31/2012-03-31 20:00:01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2012-03/2012-03-26/2012-03-31/2012-03-31 21:00:01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2012-03/2012-03-26/2012-03-31/2012-03-31 22:00:01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [280]:
S_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2294 entries, 2012-01 to 2012-03/2012-03-26/2012-03-31/2012-03-31 23:00:01
Columns: 2184 entries, 2012-01/2011-12-26/2012-01-01/2012-01-01 00:00:01 to 2012-03/2012-03-26/2012-03-31/2012-03-31 23:00:01
dtypes: float32(2184)
memory usage: 19.1+ MB


In [281]:
Y_hier_df

,unique_id,ds,y
0,2012-01,2012-01-01 00:00:01,69.0
1,2012-01,2012-01-01 01:00:01,92.0
2,2012-01,2012-01-01 02:00:01,96.0
3,2012-01,2012-01-01 03:00:01,92.0
4,2012-01,2012-01-01 04:00:01,91.0
...,...,...,...
8731,2012-03/2012-03-26/2012-03-31/2012-03-31 19:00:01,2012-03-31 19:00:01,110.0
8732,2012-03/2012-03-26/2012-03-31/2012-03-31 20:00:01,2012-03-31 20:00:01,116.0
8733,2012-03/2012-03-26/2012-03-31/2012-03-31 21:00:01,2012-03-31 21:00:01,105.0
8734,2012-03/2012-03-26/2012-03-31/2012-03-31 22:00:01,2012-03-31 22:00:01,95.0


## We have the hierarchical setup ready, so let us build a set of baseline forecasts: we will use the last 7 days as test set.

In [282]:
FH=7

In [283]:
# Split train/test sets
Y_test_df  = Y_hier_df.groupby('unique_id').tail(FH)
Y_train_df = Y_hier_df.drop(Y_test_df.index)

Y_test_df = Y_test_df.set_index('unique_id')
Y_train_df = Y_train_df.set_index('unique_id')

Y_train_df.info(), Y_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5782 entries, 2012-01 to 2012-03/2012-03-26/2012-03-31
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      5782 non-null   datetime64[ns]
 1   y       5782 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 135.5+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 2954 entries, 2012-01 to 2012-03/2012-03-26/2012-03-31/2012-03-31 23:00:01
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      2954 non-null   datetime64[ns]
 1   y       2954 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 69.2+ KB


(None, None)

In [284]:
Y_train_df

,ds,y
unique_id,,
2012-01,2012-01-01 00:00:01,69.0
2012-01,2012-01-01 01:00:01,92.0
2012-01,2012-01-01 02:00:01,96.0
2012-01,2012-01-01 03:00:01,92.0
2012-01,2012-01-01 04:00:01,91.0
...,...,...
2012-03/2012-03-26/2012-03-31,2012-03-31 12:00:01,128.0
2012-03/2012-03-26/2012-03-31,2012-03-31 13:00:01,127.0
2012-03/2012-03-26/2012-03-31,2012-03-31 14:00:01,120.0


In [285]:
Y_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5782 entries, 2012-01 to 2012-03/2012-03-26/2012-03-31
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      5782 non-null   datetime64[ns]
 1   y       5782 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 135.5+ KB


In [286]:
# Создаем новый столбец со старым индексом
Y_test_df['unique_id'] = Y_test_df.index

# Сбрасываем индекс и заменяем его числовым индексом от 0 до длины датафрейма
Y_test_df.reset_index(drop=True, inplace=True)

# Устанавливаем новый индекс обратно на старый индекс
# Y_test_df.set_index('unique_id', inplace=True)
Y_test_df

,ds,y,unique_id
0,2012-01-31 17:00:01,105.0,2012-01
1,2012-01-31 18:00:01,110.0,2012-01
2,2012-01-31 19:00:01,121.0,2012-01
3,2012-01-31 20:00:01,133.0,2012-01
4,2012-01-31 21:00:01,115.0,2012-01
...,...,...,...
2949,2012-03-31 19:00:01,110.0,2012-03/2012-03-26/2012-03-31/2012-03-31 19:00:01
2950,2012-03-31 20:00:01,116.0,2012-03/2012-03-26/2012-03-31/2012-03-31 20:00:01
2951,2012-03-31 21:00:01,105.0,2012-03/2012-03-26/2012-03-31/2012-03-31 21:00:01
2952,2012-03-31 22:00:01,95.0,2012-03/2012-03-26/2012-03-31/2012-03-31 22:00:01


In [287]:
# Создаем новый столбец со старым индексом
Y_train_df['unique_id'] = Y_train_df.index

# Сбрасываем индекс и заменяем его числовым индексом от 0 до длины датафрейма
Y_train_df.reset_index(drop=True, inplace=True)
Y_train_df

,ds,y,unique_id
0,2012-01-01 00:00:01,69.0,2012-01
1,2012-01-01 01:00:01,92.0,2012-01
2,2012-01-01 02:00:01,96.0,2012-01
3,2012-01-01 03:00:01,92.0,2012-01
4,2012-01-01 04:00:01,91.0,2012-01
...,...,...,...
5777,2012-03-31 12:00:01,128.0,2012-03/2012-03-26/2012-03-31
5778,2012-03-31 13:00:01,127.0,2012-03/2012-03-26/2012-03-31
5779,2012-03-31 14:00:01,120.0,2012-03/2012-03-26/2012-03-31
5780,2012-03-31 15:00:01,114.0,2012-03/2012-03-26/2012-03-31


## попытка сделать предсказание без реконсиляции

In [288]:
metric = "mse"
stat_level = "month/week/day/hour"

In [289]:
Y_train_df.groupby("unique_id").tail(2), Y_test_df.groupby("unique_id").tail(2)

(                      ds      y                      unique_id
 735  2012-01-31 15:00:01  100.0                        2012-01
 736  2012-01-31 16:00:01  103.0                        2012-01
 1424 2012-02-29 15:00:01  106.0                        2012-02
 1425 2012-02-29 16:00:01  161.0                        2012-02
 2161 2012-03-31 15:00:01  114.0                        2012-03
 ...                  ...    ...                            ...
 5747 2012-03-29 16:00:01  107.0  2012-03/2012-03-26/2012-03-29
 5763 2012-03-30 15:00:01   99.0  2012-03/2012-03-26/2012-03-30
 5764 2012-03-30 16:00:01   99.0  2012-03/2012-03-26/2012-03-30
 5780 2012-03-31 15:00:01  114.0  2012-03/2012-03-26/2012-03-31
 5781 2012-03-31 16:00:01  103.0  2012-03/2012-03-26/2012-03-31
 
 [220 rows x 3 columns],
                       ds      y  \
 5    2012-01-31 22:00:01  105.0   
 6    2012-01-31 23:00:01   94.0   
 12   2012-02-29 22:00:01  109.0   
 13   2012-02-29 23:00:01   97.0   
 19   2012-03-31 22:00:01

In [290]:
import numpy as np
import pandas as pd

# compute base forecast no coherent
from statsforecast.models import (
    Naive,
    AutoARIMA,
    HoltWinters,
    CrostonClassic as Croston, 
    HistoricAverage,
    DynamicOptimizedTheta as DOT,
    SeasonalNaive,
    ETS,
    IMAPA,
    RandomWalkWithDrift,
    SeasonalExponentialSmoothing,
    SeasonalWindowAverage,
    SimpleExponentialSmoothing,
    TSB,
    WindowAverage,
    DynamicOptimizedTheta,
    AutoETS,
    AutoCES
)
from statsforecast.core import StatsForecast

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.utils import aggregate
from hierarchicalforecast.methods import BottomUp, TopDown
from hierarchicalforecast.core import HierarchicalReconciliation

In [291]:
# Compute base Naive predictions
# Careful identifying correct data freq, this data quarterly 'Q'
# NG: Changed freq from Q to M since inut data is monthly
# NG: Changed model from Naive to ARIMA since Naive output is already reconciled without doing anything
SP = 5  
fcst = StatsForecast(df=Y_train_df,
                     models=[AutoARIMA(season_length=SP), AutoETS(season_length=SP)],
                     freq='H', n_jobs=-1)

/home/olga/hse_master/develop_tools/fast_api/fast_api_tutorial/venv/lib/python3.10/site-packages/statsforecast/core.py:399: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(


In [292]:
Y_hat_df = fcst.forecast(h=FH, fitted=True)
Y_fitted_df = fcst.forecast_fitted_values()

/home/olga/hse_master/develop_tools/fast_api/fast_api_tutorial/venv/lib/python3.10/site-packages/statsforecast/core.py:399: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
/home/olga/hse_master/develop_tools/fast_api/fast_api_tutorial/venv/lib/python3.10/site-packages/statsforecast/core.py:417: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
/home/olga/hse_master/develop_tools/fast_api/fast_api_tutorial/venv/lib/python3.10/site-packages/statsforecast/core.py:417: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable t

In [293]:
Y_hat_df.groupby("unique_id").tail(2), Y_fitted_df.groupby("unique_id").tail(2)

(                                               ds   AutoARIMA     AutoETS
 unique_id                                                                
 2012-01                       2012-01-31 22:00:01  102.269241  102.999702
 2012-01                       2012-01-31 23:00:01  101.871391  102.999702
 2012-01/2011-12-26            2012-01-01 22:00:01   83.366928  109.000298
 2012-01/2011-12-26            2012-01-01 23:00:01   83.274742  109.000298
 2012-01/2011-12-26/2012-01-01 2012-01-01 22:00:01   83.366928  109.000298
 ...                                           ...         ...         ...
 2012-03/2012-03-26/2012-03-29 2012-03-29 23:00:01   96.397438  106.996399
 2012-03/2012-03-26/2012-03-30 2012-03-30 22:00:01   99.000000   99.000000
 2012-03/2012-03-26/2012-03-30 2012-03-30 23:00:01   99.000000   99.000000
 2012-03/2012-03-26/2012-03-31 2012-03-31 22:00:01   80.211235  103.001099
 2012-03/2012-03-26/2012-03-31 2012-03-31 23:00:01   79.289597  103.001099
 
 [220 rows x 3 columns]

In [294]:
Y_hat_df

,ds,AutoARIMA,AutoETS
unique_id,,,
2012-01,2012-01-31 17:00:01,102.592194,102.999702
2012-01,2012-01-31 18:00:01,102.737717,102.999702
2012-01,2012-01-31 19:00:01,103.324272,102.999702
2012-01,2012-01-31 20:00:01,103.779823,102.999702
2012-01,2012-01-31 21:00:01,102.951683,102.999702
...,...,...,...
2012-03/2012-03-26/2012-03-31,2012-03-31 19:00:01,86.063492,103.001099
2012-03/2012-03-26/2012-03-31,2012-03-31 20:00:01,83.396271,103.001099
2012-03/2012-03-26/2012-03-31,2012-03-31 21:00:01,81.524620,103.001099


In [295]:
Y_fitted_df

,ds,y,AutoARIMA,AutoETS
unique_id,,,,
2012-01,2012-01-01 00:00:01,69.0,60.922459,69.123726
2012-01,2012-01-01 01:00:01,92.0,69.693207,69.000015
2012-01,2012-01-01 02:00:01,96.0,94.830650,91.997704
2012-01,2012-01-01 03:00:01,92.0,91.101509,95.999603
2012-01,2012-01-01 04:00:01,91.0,90.194138,92.000397
...,...,...,...,...
2012-03/2012-03-26/2012-03-31,2012-03-31 12:00:01,128.0,128.403442,126.999802
2012-03/2012-03-26/2012-03-31,2012-03-31 13:00:01,127.0,128.701721,127.999901
2012-03/2012-03-26/2012-03-31,2012-03-31 14:00:01,120.0,126.298279,127.000099


In [296]:
# You can select a reconciler from our collection
reconcilers = [BottomUp()] # MinTrace(method='mint_shrink')
hrec = HierarchicalReconciliation(reconcilers=reconcilers)

In [297]:
Y_rec_df = hrec.reconcile(Y_h=Y_hat_df, 
                          Y_df=Y_fitted_df,
                          S=S_df, tags=tags)
Y_rec_df.groupby('unique_id').head(2)

ValueError: Length of values (70070) does not match length of index (770)

In [299]:
S_df

,2012-01/2011-12-26/2012-01-01/2012-01-01 00:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 01:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 02:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 03:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 04:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 05:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 06:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 07:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 08:00:01,2012-01/2011-12-26/2012-01-01/2012-01-01 09:00:01,...,2012-03/2012-03-26/2012-03-31/2012-03-31 14:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 15:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 16:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 17:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 18:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 19:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 20:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 21:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 22:00:01,2012-03/2012-03-26/2012-03-31/2012-03-31 23:00:01
2012-01,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2012-01/2011-12-26,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-01/2012-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-03/2012-03-26/2012-03-31/2012-03-31 19:00:01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2012-03/2012-03-26/2012-03-31/2012-03-31 20:00:01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2012-03/2012-03-26/2012-03-31/2012-03-31 21:00:01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2012-03/2012-03-26/2012-03-31/2012-03-31 22:00:01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [300]:
S_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2294 entries, 2012-01 to 2012-03/2012-03-26/2012-03-31/2012-03-31 23:00:01
Columns: 2184 entries, 2012-01/2011-12-26/2012-01-01/2012-01-01 00:00:01 to 2012-03/2012-03-26/2012-03-31/2012-03-31 23:00:01
dtypes: float32(2184)
memory usage: 19.2+ MB


In [301]:
Y_hat_df

,ds,AutoARIMA,AutoETS
unique_id,,,
2012-01,2012-01-31 17:00:01,102.592194,102.999702
2012-01,2012-01-31 18:00:01,102.737717,102.999702
2012-01,2012-01-31 19:00:01,103.324272,102.999702
2012-01,2012-01-31 20:00:01,103.779823,102.999702
2012-01,2012-01-31 21:00:01,102.951683,102.999702
...,...,...,...
2012-03/2012-03-26/2012-03-31,2012-03-31 19:00:01,86.063492,103.001099
2012-03/2012-03-26/2012-03-31,2012-03-31 20:00:01,83.396271,103.001099
2012-03/2012-03-26/2012-03-31,2012-03-31 21:00:01,81.524620,103.001099


In [302]:
Y_hat_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 770 entries, 2012-01 to 2012-03/2012-03-26/2012-03-31
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   ds         770 non-null    datetime64[ns]
 1   AutoARIMA  770 non-null    float32       
 2   AutoETS    770 non-null    float32       
dtypes: datetime64[ns](1), float32(2)
memory usage: 34.2+ KB


In [303]:
Y_fitted_df

,ds,y,AutoARIMA,AutoETS
unique_id,,,,
2012-01,2012-01-01 00:00:01,69.0,60.922459,69.123726
2012-01,2012-01-01 01:00:01,92.0,69.693207,69.000015
2012-01,2012-01-01 02:00:01,96.0,94.830650,91.997704
2012-01,2012-01-01 03:00:01,92.0,91.101509,95.999603
2012-01,2012-01-01 04:00:01,91.0,90.194138,92.000397
...,...,...,...,...
2012-03/2012-03-26/2012-03-31,2012-03-31 12:00:01,128.0,128.403442,126.999802
2012-03/2012-03-26/2012-03-31,2012-03-31 13:00:01,127.0,128.701721,127.999901
2012-03/2012-03-26/2012-03-31,2012-03-31 14:00:01,120.0,126.298279,127.000099


In [304]:
from sklearn.metrics import mean_squared_error as mse

In [305]:
# Вычисляем MSE для AutoARIMA
mse_autoarima = mse(Y_fitted_df['y'], Y_fitted_df['AutoARIMA'])

# Вычисляем MSE для AutoETS
mse_autoets = mse(Y_fitted_df['y'], Y_fitted_df['AutoETS'])

print(f"MSE для AutoARIMA: {mse_autoarima}")
print(f"MSE для AutoETS: {mse_autoets}")

MSE для AutoARIMA: 177.22064208984375
MSE для AutoETS: 201.31861877441406


In [306]:
from sklearn.metrics import mean_absolute_error

# Фактические и прогнозные значения
actual_values = Y_fitted_df['y']
predicted_values = Y_fitted_df['AutoARIMA']  # или другой столбец с прогнозными значениями

# Вычисление MAE
mae = mean_absolute_error(actual_values, predicted_values)
print("MAE:", mae)

MAE: 7.881965


In [307]:
Y_fitted_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5782 entries, 2012-01 to 2012-03/2012-03-26/2012-03-31
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   ds         5782 non-null   datetime64[ns]
 1   y          5782 non-null   float32       
 2   AutoARIMA  5782 non-null   float32       
 3   AutoETS    5782 non-null   float32       
dtypes: datetime64[ns](1), float32(3)
memory usage: 287.1+ KB


In [308]:
tags

{'month': array(['2012-01', '2012-02', '2012-03'], dtype=object),
 'month/week': array(['2012-01/2011-12-26', '2012-01/2012-01-02', '2012-01/2012-01-09',
        '2012-01/2012-01-16', '2012-01/2012-01-23', '2012-01/2012-01-30',
        '2012-02/2012-01-30', '2012-02/2012-02-06', '2012-02/2012-02-13',
        '2012-02/2012-02-20', '2012-02/2012-02-27', '2012-03/2012-02-27',
        '2012-03/2012-03-05', '2012-03/2012-03-12', '2012-03/2012-03-19',
        '2012-03/2012-03-26'], dtype=object),
 'month/week/day': array(['2012-01/2011-12-26/2012-01-01', '2012-01/2012-01-02/2012-01-02',
        '2012-01/2012-01-02/2012-01-03', '2012-01/2012-01-02/2012-01-04',
        '2012-01/2012-01-02/2012-01-05', '2012-01/2012-01-02/2012-01-06',
        '2012-01/2012-01-02/2012-01-07', '2012-01/2012-01-02/2012-01-08',
        '2012-01/2012-01-09/2012-01-09', '2012-01/2012-01-09/2012-01-10',
        '2012-01/2012-01-09/2012-01-11', '2012-01/2012-01-09/2012-01-12',
        '2012-01/2012-01-09/2012-01-13', '

In [309]:
tags.info()

AttributeError: 'dict' object has no attribute 'info'

In [ ]:
# You can select a reconciler from our collection
reconcilers = [
      BottomUp(),
      TopDown(method='forecast_proportions'),
      # TopDown(method='average_proportions'),
      # TopDown(method='proportion_averages'),
      MinTrace(method='ols'),
      # MinTrace(method='wls_var'),
      # MinTrace(method='mint_shrink'),
      # #ERM(method='reg_bu', lambda_reg=100) # Extremely inneficient
      ERM(method='closed')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)

In [310]:
Y_rec_df = hrec.reconcile(Y_h=Y_hat_df, 
                          Y_df=Y_fitted_df,
                          S=S_df, tags=tags)
Y_rec_df.groupby('unique_id').head(FH)

ValueError: Length of values (70070) does not match length of index (770)

In [311]:
Y_hat_df.shape

(770, 3)

In [312]:
Y_hat_df.index.shape

(770,)